## Usando K-means para compactação de cores

Observe que este notebook foi concebido como um exemplo demonstrativo de como o algoritmo K-means funciona. Seu foco está na teoria e nos resultados de sua aplicação, e não no código em si. 
Este laboratório irá:
- Dar a você um exemplo prático do K-means em dados não sintéticos
- Reforçar sua compreensão da teoria subjacente do K-means demonstrando seu efeito quando aplicado a uma fotografia

Objetivo da modelagem
Usaremos o K-means para agrupar os pixels de uma fotografia de algumas tulipas com base em seus valores de cor codificados. Exploraremos como diferentes valores de k afetam o agrupamento dos pixels e, portanto, a aparência da fotografia. Também examinaremos o que está acontecendo “nos bastidores” durante a execução do algoritmo.

Declarações de importação
Usaremos o numpy e o pandas para operações e o Plotly para visualização em 3D. Um destaque especial é o Kmeans, que é a implementação do algoritmo K-means pelo scikit-learn.

In [ ]:
import numpy as np
import pandas as pd

%pylab inline
import plotly.graph_objects as go

from sklearn.cluster import KMeans

Leitura de dados
Os “dados”, neste caso, não são um dataframe do pandas. É uma fotografia, que converteremos em uma matriz numérica.

In [ ]:
# Read in a photo
img = plt.imread('using_kmeans_for_color_compression_tulips_photo.jpg')

In [ ]:
# Exibir a foto e seu formato
print(img.shape)
plt.imshow(img)
plt.axis('off');

Aqui temos uma fotografia de algumas tulipas. O formato da imagem é 320 x 240 x 3. Isso pode ser interpretado como informação de pixel. Cada ponto na sua tela é um pixel. Essa fotografia tem 320 pixels verticais e 240 pixels horizontais.

In [ ]:
# Remodelar a imagem de modo que cada linha represente um único pixel
# definido por três valores: R, G, B
img_flat = img.reshape(img.shape[0]*img.shape[1], 3)
img_flat[:5, :]

In [ ]:
img_flat.shape

Agora temos uma matriz de 76.800 x 3. Cada linha representa os valores de cor de um único pixel. Como temos apenas 3 colunas, podemos visualizar esses dados em um espaço tridimensional. Vamos criar um dataframe do pandas para nos ajudar a entender e visualizar os dados.

In [ ]:
# Criar um pandas df com r, g e b como colunas
img_flat_df = pd.DataFrame(img_flat, columns = ['r', 'g', 'b'])
img_flat_df.head()

Observação: A saída da célula a seguir pode ser visualizada de duas maneiras: Você pode executar novamente essa célula ou converter manualmente o notebook para “Confiável”.

In [ ]:
# Crie um gráfico 3D em que cada pixel da imagem seja exibido em sua cor real
trace = go.Scatter3d(x = img_flat_df.r,
                     y = img_flat_df.g,
                     z = img_flat_df.b,
                     mode='markers',
                     marker=dict(size=1,
                                 color=['rgb({},{},{})'.format(r,g,b) for r,g,b 
                                        in zip(img_flat_df.r.values, 
                                               img_flat_df.g.values, 
                                               img_flat_df.b.values)],
                                 opacity=0.5))

data = [trace]

layout = go.Layout(margin=dict(l=0,
                               r=0,
                               b=0,
                               t=0),
                               )

fig = go.Figure(data=data, layout=layout)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'),
                  )
fig.show()

Nesse gráfico, cada ponto representa uma cor/pixel que está em nossa imagem original de tulipas. Quanto mais intensa for a cor, mais pontos estarão concentrados nessa área. As cores mais representadas no gráfico são as cores mais abundantes na fotografia: principalmente vermelhos, verdes e amarelos. Você pode clicar e girar esse gráfico para examiná-lo de diferentes ângulos. Também é possível aumentar e diminuir o zoom.

Podemos treinar um modelo K-means com esses dados. O algoritmo criará k clusters minimizando as distâncias quadradas de cada ponto até o centroide mais próximo.

Vamos primeiro fazer um experimento. Construir um modelo K-means com apenas um único centroide (k = 1) e substituíssemos cada pixel da fotografia pelo valor RGB desse centroide? Qual seria a aparência da fotografia?

Agrupar os dados: k = 1

In [ ]:
# Instanciar o modelo
kmeans = KMeans(n_clusters=1, random_state=42).fit(img_flat)

In [ ]:
# Copie o `img_flat` para que possamos modificá-lo
img_flat1 = img_flat.copy()

# Substitua cada linha da imagem original pelo centro do cluster mais próximo
for i in np.unique(kmeans.labels_):
    img_flat1[kmeans.labels_==i,:] = kmeans.cluster_centers_[i]

# Remodelar os dados de volta para (640, 480, 3)
img1 = img_flat1.reshape(img.shape)

plt.imshow(img1)
plt.axis('off');

Então, o que aconteceu? Bem, vamos percorrer as etapas do K-means:

Colocamos nosso centroide aleatoriamente no espaço de cores.
Atribuímos cada ponto ao seu centroide mais próximo. Como havia apenas um centroide, todos os pontos foram atribuídos ao mesmo centroide e, portanto, ao mesmo cluster.
Atualizamos a localização do centroide para a localização média de todos os seus pontos. Novamente, como há apenas um único centroide, ele foi atualizado para o local médio de cada ponto na imagem.
Repita até o modelo convergir. Nesse caso, foi necessária apenas uma iteração para que o modelo convergisse.
Em seguida, atualizamos os valores RGB de cada pixel para que fossem iguais aos do centroide. O resultado é a imagem de nossas tulipas quando cada pixel é substituído pela cor média. A cor média dessa foto era marrom - todas as cores estavam misturadas.

Podemos verificar isso por nós mesmos calculando manualmente a média de cada coluna na matriz achatada. Isso nos dará o valor R médio, o valor G e o valor B.

In [ ]:
# Calcular a média de cada coluna na matriz achatada
column_means = img_flat.mean(axis=0)

print('column means: ', column_means)

In [ ]:
print('cluster centers: ', kmeans.cluster_centers_)

Eles são iguais! Agora, vamos voltar à renderização 3D de nossos dados, só que desta vez adicionaremos o centroide.

In [ ]:
# Crie um gráfico 3-D em que cada pixel da imagem seja exibido em sua cor real
trace = go.Scatter3d(x = img_flat_df.r,
                     y = img_flat_df.g,
                     z = img_flat_df.b,
                     mode='markers',
                     marker=dict(size=1,
                                 color=['rgb({},{},{})'.format(r,g,b) for 
                                        r,g,b in zip(img_flat_df.r.values, 
                                                     img_flat_df.g.values, 
                                                     img_flat_df.b.values)],
                                 opacity=0.5))

data = [trace]

layout = go.Layout(margin=dict(l=0,
                               r=0,
                               b=0,
                               t=0))

fig = go.Figure(data=data, layout=layout)


# Adicionar centroide ao gráfico
centroid = kmeans.cluster_centers_[0].tolist()

fig.add_trace(
    go.Scatter3d(x = [centroid[0]],
                 y = [centroid[1]],
                 z = [centroid[2]],
                 mode='markers',
                 marker=dict(size=7,
                             color=['rgb(125.79706706,78.8178776,42.58090169)'],
                             opacity=1))
)
fig.update_layout(scene = dict(
                    xaxis_title='R',
                    yaxis_title='G',
                    zaxis_title='B'),
                  )
fig.show()

Podemos ver o centroide como um grande círculo no meio do espaço de cores. (Se não conseguir, clique na imagem e gire/faça zoom.) Observe que esse é o “centro de gravidade” de todos os pontos no gráfico.

Agora vamos tentar outra coisa. Vamos reajustar um modelo K-means aos dados, desta vez usando k = 3. Reserve um momento para considerar o que você pode esperar como resultado disso. Percorra as etapas do que o modelo está fazendo, como fizemos acima. Que cores você provavelmente verá?

Agrupar os dados: k = 3

In [ ]:
# Instanciar o modelo k-means para 3 clusters
kmeans3 = KMeans(n_clusters=3, random_state=42).fit(img_flat)

# Verifique os valores exclusivos do que é retornado pelo atributo .labels_  
np.unique(kmeans3.labels_)

O atributo .cluster_centers_ retorna uma matriz em que cada elemento representa as coordenadas de um centroide (ou seja, seus valores RGB). Usaremos essas coordenadas como fizemos anteriormente para gerar as cores representadas por nossos centroides.

In [ ]:
# Atribuir coordenadas do centroide à variável `centers`.
centers = kmeans3.cluster_centers_
centers

Agora, criaremos uma função auxiliar para exibir facilmente valores RGB como amostras de cores e usá-la para verificar as cores dos centroides do modelo.

In [ ]:
# Função auxiliar que cria amostras de cores
def show_swatch(RGB_value):
    '''
    Takes in an RGB value and outputs a color swatch
    '''
    R, G, B = RGB_value
    rgb = [[np.array([R,G,B]).astype('uint8')]]
    plt.figure()
    plt.imshow(rgb)
    plt.axis('off');

In [ ]:
# Exibir as amostras de cores
for pixel in centers:
    show_swatch(pixel)

Esperamos que você tenha levantado a hipótese de que veríamos cores semelhantes como resultado de um modelo de 3 clusters. Se você examinar a imagem original das tulipas, verá que geralmente há três cores dominantes: vermelhos, verdes e dourados/amarelos, o que é muito próximo do que o modelo retornou.

Da mesma forma que antes, vamos substituir cada pixel da imagem original pelo valor RGB do centroide ao qual ele foi atribuído.

In [ ]:
# Função auxiliar para exibir nossa fotografia quando agrupada em k clusters
def cluster_image(k, img=img):

    img_flat = img.reshape(img.shape[0]*img.shape[1], 3)
    kmeans = KMeans(n_clusters = k, random_state = 42).fit(img_flat)
    new_img = img_flat.copy()
  
    for i in np.unique(kmeans.labels_):
        new_img[kmeans.labels_ == i, :] = kmeans.cluster_centers_[i]
  
    new_img = new_img.reshape(img.shape)

    return plt.imshow(new_img), plt.axis('off');

In [ ]:
# Gerar imagem quando k=3
cluster_image(3);

Agora temos uma foto com apenas três cores. Os valores RGB de cada pixel correspondem aos valores de seu centroide mais próximo.

Podemos retornar mais uma vez ao nosso espaço de cores 3-D. Desta vez, vamos colorir novamente cada ponto no espaço de cores para corresponder à cor de seu centroide. Isso nos permitirá ver como o algoritmo K-means agrupou nossos dados espacialmente.

Novamente, não se preocupe tanto com o código. Sinta-se à vontade para pular para o gráfico.

In [ ]:
# Apenas para ter uma ideia de como são as estruturas de dados

print(kmeans3.labels_.shape)
print(kmeans3.labels_)
print(np.unique(kmeans3.labels_))
print(kmeans3.cluster_centers_)

In [ ]:
# Crie uma nova coluna no df que indique o número do cluster de cada linha  
# (conforme atribuído pelo Kmeans para k=3)
img_flat_df['cluster'] = kmeans3.labels_
img_flat_df.head()

In [ ]:
# Criar um dicionário auxiliar para mapear valores de cores RGB para cada observação em df
series_conversion = {
    0: 'rgb' + str(tuple(map(int, kmeans3.cluster_centers_[0]))),
    1: 'rgb' + str(tuple(map(int, kmeans3.cluster_centers_[1]))),
    2: 'rgb' + str(tuple(map(int, kmeans3.cluster_centers_[2]))),
}

In [ ]:
# Substitua os números de cluster na coluna 'cluster' por valores RGB formatados
# (preparado para a plotagem)
img_flat_df['cluster'] = img_flat_df['cluster'].map(series_conversion)
img_flat_df.head()

In [ ]:
# Replotar os dados, agora mostrando a qual cluster (ou seja, cor) eles foram atribuídos pelo K-means quando k=3

trace = go.Scatter3d(x = img_flat_df.r,
                     y = img_flat_df.g,
                     z = img_flat_df.b,
                     mode='markers',
                     marker=dict(size=1,
                                 color=img_flat_df.cluster,
                                 opacity=1))

data = trace

layout = go.Layout(margin=dict(l=0,
                               r=0,
                               b=0,
                               t=0))

fig = go.Figure(data=data, layout=layout)
fig.show()

Agrupar os dados: k = 2-10

Você pode estar pensando que teria agrupado os dados de forma diferente com base na distribuição de pontos que viu no primeiro gráfico 3D. Por exemplo, por que há uma linha nítida que separa o vermelho do verde, quando não parece haver nenhum espaço vazio nos dados?

Você não está incorreto. Embora não exista um agrupamento “errado”, algumas formas podem ser definitivamente melhores do que outras.

Você notará na renderização original em 3D que há faixas longas - não bolas redondas - de dados agrupados. O K-means funciona melhor quando os clusters são mais circulares, pois tenta minimizar a distância do ponto ao centroide. Pode valer a pena tentar um algoritmo de agrupamento diferente se você quiser agrupar uma faixa longa, estreita e contínua de dados

No entanto, o K-means comprime com sucesso as cores dessa fotografia. Esse processo pode ser aplicado para qualquer valor de k. Aqui está o resultado de cada foto para k = 2-10.

In [ ]:
# Função auxiliar para plotar a grade da imagem
def cluster_image_grid(k, ax, img=img):
 
    img_flat = img.reshape(img.shape[0]*img.shape[1], 3)
    kmeans = KMeans(n_clusters=k, random_state=42).fit(img_flat)
    new_img = img_flat.copy()

    for i in np.unique(kmeans.labels_):
        new_img[kmeans.labels_==i, :] = kmeans.cluster_centers_[i]

    new_img = new_img.reshape(img.shape)
    ax.imshow(new_img)
    ax.axis('off')

fig, axs = plt.subplots(3, 3)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(9, 12)
axs = axs.flatten()
k_values = np.arange(2, 11)
for i, k in enumerate(k_values):
    cluster_image_grid(k, axs[i], img=img)
    axs[i].title.set_text('k=' + str(k))

Observe que fica cada vez mais difícil ver a diferença entre as imagens cada vez que uma cor é adicionada. Esse é um exemplo visual de algo que acontece com todos os modelos de agrupamento, mesmo que os dados não sejam uma imagem que você possa ver. À medida que você agrupa os dados em mais e mais clusters, os clusters adicionais além de um determinado ponto contribuem cada vez menos para a compreensão dos dados.